In [79]:
import numpy as np # linear algebra
import pandas as pd # data processing/manipulation
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LinearRegression

from collections import Counter
import nltk
import string
from nltk.corpus import stopwords



In [80]:
#Importing Data
data = pd.read_csv('1429_1.csv')
data.head()

/tmp/ipykernel_11866/1950577414.py:2: DtypeWarning: Columns (1,10) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('1429_1.csv')


,id,name,asins,brand,categories,keys,manufacturer,reviews.date,reviews.dateAdded,reviews.dateSeen,...,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.userCity,reviews.userProvince,reviews.username
0,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,This product so far has not disappointed. My c...,Kindle,NaN,NaN,Adapter
1,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,great for beginner or experienced person. Boug...,very fast,NaN,NaN,truman
2,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,Inexpensive tablet for him to use and learn on...,Beginner tablet for our 9 year old son.,NaN,NaN,DaveZ
3,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,4.0,http://reviews.bestbuy.com/3545/5620406/review...,I've had my Fire HD 8 two weeks now and I love...,Good!!!,NaN,NaN,Shacks
4,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-12T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,I bought this for my grand daughter when she c...,Fantastic Tablet for kids,NaN,NaN,explore42


In [81]:
review=pd.DataFrame(data.groupby('reviews.rating').size().sort_values(ascending=False).rename('No of Users').reset_index())
review.head()

,reviews.rating,No of Users
0,5.0,23775
1,4.0,8541
2,3.0,1499
3,1.0,410
4,2.0,402


In [82]:
#Though the four columns below contains very less missing values, it would do no harm in removing NA values.
data1 = data[['reviews.rating' , 'reviews.text' , 'reviews.title' , 'reviews.username']]
final = data1.dropna()
final.head()

,reviews.rating,reviews.text,reviews.title,reviews.username
0,5.0,This product so far has not disappointed. My c...,Kindle,Adapter
1,5.0,great for beginner or experienced person. Boug...,very fast,truman
2,5.0,Inexpensive tablet for him to use and learn on...,Beginner tablet for our 9 year old son.,DaveZ
3,4.0,I've had my Fire HD 8 two weeks now and I love...,Good!!!,Shacks
4,5.0,I bought this for my grand daughter when she c...,Fantastic Tablet for kids,explore42


In [83]:
#No NA values
fact =  final[final["reviews.text"].isnull()]
fact.head()

,reviews.rating,reviews.text,reviews.title,reviews.username


In [138]:
rating = final[(final['reviews.rating'] == 1.0) | (final['reviews.rating'] == 2.0) | (final['reviews.rating'] == 3.0) | (final['reviews.rating'] == 5)]
# print(rating['reviews.rating'].value_counts())
rating['reviews.rating'] = rating['reviews.rating'].replace([1.0, 2.0, 3.0, 5.0],[1.0,1.0,1.0,5.0])
print(rating['reviews.rating'].value_counts())
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter


5.0    23768
1.0     2311
Name: reviews.rating, dtype: int64


/tmp/ipykernel_11866/3526900360.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rating['reviews.rating'] = rating['reviews.rating'].replace([1.0, 2.0, 3.0, 5.0],[1.0,1.0,1.0,5.0])


In [85]:
y = rating['reviews.rating']
X = rating['reviews.text']
X.head()

0    This product so far has not disappointed. My c...
1    great for beginner or experienced person. Boug...
2    Inexpensive tablet for him to use and learn on...
4    I bought this for my grand daughter when she c...
5    This amazon fire 8 inch tablet is the perfect ...
Name: reviews.text, dtype: object

In [86]:
import nltk
from nltk.corpus import stopwords

print(stopwords.words('english'))
# stop_words = set(stopwords.words('english'))


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [87]:

nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/ggdv05/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [88]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/ggdv05/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [89]:
#Stemming
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
  
 
stemmer = PorterStemmer() 
# lemmatize string 
def stemmerize_word(text): 
    word_tokens = word_tokenize(text) 
    # provide context i.e. part-of-speech 
    lemmas = [stemmer.stem(word) for word in word_tokens] 
    return lemmas 
  
text = 'Inexpensive tablet for him to use and learn on, step up from the NABI. He was thrilled with it, learn how to Skype on it already....'
stemmerize_word(text) 

['inexpens',
 'tablet',
 'for',
 'him',
 'to',
 'use',
 'and',
 'learn',
 'on',
 ',',
 'step',
 'up',
 'from',
 'the',
 'nabi',
 '.',
 'he',
 'wa',
 'thrill',
 'with',
 'it',
 ',',
 'learn',
 'how',
 'to',
 'skype',
 'on',
 'it',
 'alreadi',
 '....']

In [90]:
import string
stop_words=set(stopwords.words('english'))
def text_process(text):
    # Removing all the characters except alphabets and numbers
    nopunc = [char for char in text if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    words=[word for word in nopunc.split() if word.lower() not in stop_words]
    s = ' '.join(words)
    return stemmerize_word(s)

In [91]:
sample_text = "Inexpensive tablet for him to use and learn on, step up from the NABI. He was thrilled with it, learn how to Skype on it already...."
print(text_process(sample_text))

['inexpens', 'tablet', 'use', 'learn', 'step', 'nabi', 'thrill', 'learn', 'skype', 'alreadi']


Using CountVectorizer and vectorizer to return a vector array.

In [92]:
from sklearn.feature_extraction.text import CountVectorizer

transformer = CountVectorizer(analyzer=text_process).fit(X)

In [49]:
print(transformer)

CountVectorizer(analyzer=<function text_process at 0x7f04746150d0>)


In [93]:
X = transformer.transform(X)

In [94]:
#Lets start training the model
from sklearn.model_selection import train_test_split
#using 30% of the data for testing, this will be revised once we do not get the desired accuracy
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

In [95]:
from sklearn.naive_bayes import MultinomialNB
naive = MultinomialNB()
naive.fit(X_train, y_train)

MultinomialNB()

In [96]:
pred = naive.predict(X_test)

In [97]:
from sklearn.metrics import confusion_matrix, classification_report
print(confusion_matrix(y_test, pred))
print('\n')

print(classification_report(y_test, pred))
naive.score(X_train, y_train)

[[ 283  439]
 [ 139 6963]]


              precision    recall  f1-score   support

         1.0       0.67      0.39      0.49       722
         5.0       0.94      0.98      0.96      7102

    accuracy                           0.93      7824
   macro avg       0.81      0.69      0.73      7824
weighted avg       0.92      0.93      0.92      7824



0.9475760065735415

In [101]:
inputword = "terrible"

x = [text_process(inputword)]
print(x)
x = transformer.transform(x)
print(naive.predict(x))


[['terribl']]
[1.]
